In [ ]:
pip install beautifulsoup4 requests pandas

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import numpy as np

# URL for the 2017 NFL draft
draft_url = 'https://www.pro-football-reference.com/years/2017/draft.htm'

# Get the draft page
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36'
}
draft_response = requests.get(draft_url, headers=headers)

# Check the request
if draft_response.status_code != 200:
    print(f"Failed to retrieve page. Status code: {draft_response.status_code}")
else:
    # Parse the page content using BeautifulSoup
    soup = BeautifulSoup(draft_response.content, 'html.parser')

    # Find the table containing drafted players
    draft_table = soup.find('table', {'id': 'drafts'})

    # Check if the table was found
    if draft_table is None:
        print("Draft table not found on the page. Please check the page structure.")
    else:
        # Extract player URLs
        player_data = []
        for row in draft_table.find('tbody').find_all('tr'):
            # Skip header rows
            if row.get('class') and 'thead' in row.get('class'):
                continue

            player_link = row.find('td', {'data-stat': 'player'})
            if player_link and player_link.a:
                player_name = player_link.a.text
                player_url = 'https://www.pro-football-reference.com' + player_link.a['href']
                year_drafted = 2017
                pick = row.find('td', {'data-stat': 'draft_pick'}).text
                player_data.append({'name': player_name, 'url': player_url, 'year_drafted': year_drafted, 'pick': pick})

        # Function to extract AV values from all tables on a player's page
        def extract_av_values(player_url, year_drafted):
            response = requests.get(player_url, headers=headers)
            soup = BeautifulSoup(response.content, 'html.parser')

            # Initialize AV values with NaN for each year in the 5-year window
            av_values_by_year = {year: [] for year in range(year_drafted, year_drafted + 5)}

            # Find all tables in the player's page
            tables = soup.find_all('table')
            for table in tables:
                # Check if table contains AV data by searching for 'av' in data-stat attributes
                for row in table.find('tbody').find_all('tr'):
                    year_cell = row.find('th', {'data-stat': 'year_id'})
                    if year_cell:
                        year = year_cell.text.strip()
                        clean_year = ''.join(filter(str.isdigit, year))  # Remove non-numeric characters from year
                        try:
                            clean_year = int(clean_year)
                        except ValueError:
                            continue

                        # Check if the year is within the 5-year window since being drafted
                        if year_drafted <= clean_year < year_drafted + 5:
                            av_cell = row.find('td', {'data-stat': 'av'})
                            if av_cell and av_cell.text.strip().isdigit():
                                av_value = int(av_cell.text.strip())
                                av_values_by_year[clean_year].append(av_value)

            # Calculate the average AV for each year (use np.nanmean to handle empty lists)
            avg_av_values = [np.nanmean(av_values_by_year[year]) if av_values_by_year[year] else np.nan
                             for year in range(year_drafted, year_drafted + 5)]

            return avg_av_values

        # Loop through each player and get average AV values with a delay to avoid throttling
        for player in player_data:
            time.sleep(2)  # Add a 2-second delay between requests
            avg_av_values = extract_av_values(player['url'], player['year_drafted'])
            player['avg_av_values'] = avg_av_values

        # Save the data to a DataFrame
        df = pd.DataFrame(player_data)
        av_columns = ['year_1_avg_av', 'year_2_avg_av', 'year_3_avg_av', 'year_4_avg_av', 'year_5_avg_av']
        df[av_columns] = pd.DataFrame(df['avg_av_values'].tolist(), index=df.index)
        df = df.drop(columns=['avg_av_values'])  # Drop avg_av_values column after splitting into separate years

        # Handle NaN explicitly for better CSV output
        df.fillna('NaN', inplace=True)

        # Save the DataFrame to a CSV file
        df.to_csv('2017_NFL_Draft_AV_Averages.csv', index=False)
        print("Data saved to 2017_NFL_Draft_AV_Averages.csv")

        # Display results for validation
        for player in player_data[:5]:  # Displaying only the first 5 players to keep output manageable
            print(f"Name: {player['name']}, Pick: {player['pick']}, Average AV Values: {player['avg_av_values']}, URL: {player['url']}")


Data saved to 2017_NFL_Draft_AV_Averages.csv
Name: Myles Garrett, Pick: 1, Average AV Values: [4.0, 14.0, 5.0, 15.0, 17.0], URL: https://www.pro-football-reference.com/players/G/GarrMy00.htm
Name: Mitchell Trubisky, Pick: 2, Average AV Values: [8.0, 13.0, 8.0, 7.0, 0.0], URL: https://www.pro-football-reference.com/players/T/TrubMi00.htm
Name: Solomon Thomas, Pick: 3, Average AV Values: [6.0, 6.0, 3.0, 1.0, 1.0], URL: https://www.pro-football-reference.com/players/T/ThomSo00.htm
Name: Leonard Fournette, Pick: 4, Average AV Values: [8.0, 3.0, 10.0, 5.0, 9.0], URL: https://www.pro-football-reference.com/players/F/FourLe00.htm
Name: Corey Davis, Pick: 5, Average AV Values: [3.0, 7.0, 5.0, 11.0, 4.0], URL: https://www.pro-football-reference.com/players/D/DaviCo03.htm


<ipython-input-1-9e8dd1a545d2>:93: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'NaN' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna('NaN', inplace=True)


In [ ]:
import pandas as pd

# Load the 2017 NFL Draft AV Averages file
df_2017 = pd.read_csv('2017_NFL_Draft_AV_Averages.csv')

# Calculate cumulative AV values
df_2017['CUMYr1'] = df_2017['year_1_avg_av']  # Cumulative for the first year
df_2017['CUMYr2'] = df_2017[['year_1_avg_av', 'year_2_avg_av']].sum(axis=1)  # Sum of year_1 and year_2
df_2017['CUMYr3'] = df_2017[['year_1_avg_av', 'year_2_avg_av', 'year_3_avg_av']].sum(axis=1)  # Sum of year_1 to year_3
df_2017['CUMYr4'] = df_2017[['year_1_avg_av', 'year_2_avg_av', 'year_3_avg_av', 'year_4_avg_av']].sum(axis=1)  # Sum of year_1 to year_4
df_2017['CUMYr5'] = df_2017[['year_1_avg_av', 'year_2_avg_av', 'year_3_avg_av', 'year_4_avg_av', 'year_5_avg_av']].sum(axis=1)  # Sum of year_1 to year_5

# Save the result to a new CSV file
df_2017.to_csv('2017_NFL_Draft_Cumulative_AV.csv', index=False)
print("Cumulative AV values calculated and saved for 2017.")


Cumulative AV values calculated and saved for 2017.
